In [1]:
import numpy as np
import matplotlib.pyplot as plt

$$\hat{Z}_{k+1}(w)=\int_a^b \left( \underset{i}{\sum}\underset{j}{\sum}C_{p_0p_0}(w,w').G_k^i(w').(S^{-1})^{ij}.\{d_o^j - g^j(\hat{Z}_{k}) + \int_a^b  G_k^j(w^{''}).\hat{Z}_{k}(w) dw^{''} \}\right)dw'$$

where 
$$g^j(\hat{Z}_{k})=\int_a^b \text{log}\dfrac{(x^j-w)^2 + H^2}{(x^j-w)^2 + [H - \hat{Z}_{k}]^2}dw
\quad \text{,} \quad C_{p_0p_0}(w,w')= \sigma^2 \text{exp} \left\{ -\dfrac{1}{2}\dfrac{(w-w')^2}{\Delta^2}\right\}
\quad \text{,} \quad  G_k^i(w)= \dfrac{-2(H - z_k(w))}{(x^i - w)^2 + [H - z_k(w)]^2}$$ and 
$$S_k^{ij}=(C_{d_0d_0})^{ij} + \int_a^b  \left( \int_a^b  G_k^i(w').C_{p_0p_0}(w',w^{''}).G_k^j(w{''})  dw^{''}\right) dw'$$

In [2]:
Sigma= 5
Delta= 1
H= 10
d0= np.array([0.181,0.280,0.487,1.023,2.676,4.77,2.676,1.023,0.487,0.280,0.181])
C_d0_d0=0.1*np.identity(len(d0))
m= len(d0)       #for  x_i
n= 100      #for  w_i
a= 10
b= 15
nb_iter= 100
tolerance = 0.001

In [3]:
hx=(b-a)/(m-1)
hw=(b-a)/(n-1)

W = np.zeros(n)
W[0]=a
W[-1]=b

for i in range(1,n):
    W[i]= W[i-1] + hw

X = np.zeros(m)
X[0]=a
X[-1]=b

for i in range(1,m):
    X[i]= X[i-1] + hx
    
Z = np.zeros(len(W))

$$C_{p_0p_0}(w,w')= \sigma^2 \text{exp} \left\{ -\dfrac{1}{2}\dfrac{(w-w')^2}{\Delta^2}\right\}
\quad \text{,} \quad  G_k^i(w)= \dfrac{-2(H - z_k(w))}{(x^i - w)^2 + [H - z_k(w)]^2}$$

In [4]:
def C_p0_p0(w,w_prime,sig,delta):
    arg=-1/2 *((w-w_prime)**2)/(Delta**2)
    return sig**2 * np.exp(arg)


def G_k(w,x_i,Zk_w,H):
    return -2*(H-Zk_w)/((x_i-w)**2 - (H-Zk_w)**2)


Computation of
$$S_k^{ij}=(C_{d_0d_0})^{ij} + \int_a^b  \left( \int_a^b  G_k^i(w').C_{p_0p_0}(w',w^{''}).G_k^j(w{''})  dw^{''}\right) dw'=(C_{d_0d_0})^{ij} + \int_a^b  G_k^i(w') \left( \int_a^b  C_{p_0p_0}(w',w^{''}).G_k^j(w{''})  dw^{''}\right) dw'$$

In [5]:
def S_k_ij(G_k,C_d0_d0,C_p0_p0,W,X,Zk_w,H,i,j):
    result_int_1= 0
    for k in range(len(W)):
        result_int_0 = 0
        for l in range(len(W)):
            result_int_0 += C_p0_p0(W[k],W[l],Sigma,Delta)*G_k(W[l],X[j],Zk_w,H)
            
        result_int_1 += G_k(W[k],X[i],Zk_w,H)*result_int_0    
    return C_d0_d0[i,j] + result_int_1
        
     

In [6]:

# integral_sum=0
# for r in range(len(W)):      #for the last integral
#     sum_i = 0
#     for i in range(len(X)):    #for the sum i
#         sum_j = 0
#         for j in range(len(X)):     #for the sum j
            
#             sum_j += C_p0_p0(w,w_prime,sig,delta)*G_k(w,x_i,Zk_w,H)*np.linalg.inv(S)[i,j]*partial_sum
#         sum_i += sum_j
#     integral_sum += sum_i

In [7]:
# Z_k_w = 0
# Z_k1_w = 1
# while np.abs(Z_k_w - Z_k1_w) > tolerance or a < nb_iter:
#     a += 1
#     integral_sum=0
#     for r in range(len(W)):      #for the last integral
#         sum_i = 0
#         for i in range(len(X)):    #for the sum i
#             sum_j = 0
#             for j in range(len(X)):     #for the sum j

#                 sum_j += C_p0_p0(w,w_prime,sig,delta)*G_k(w,x_i,Zk_w,H)*np.linalg.inv(S)[i,j]*partial_sum
#             sum_i += sum_j
#         integral_sum += sum_i
#     Z_k_w = Z_k1_w
#     Z_k1_w = integral_sum

Computation of  $$ \{d_o^j - g^j(\hat{Z}_{k}) + \int_a^b  G_k^j(w^{''}).\hat{Z}_{k}(w) dw^{''} \}=\left\{d_o^j - \int_a^b \text{log}\dfrac{(x^j-w)^2 + H^2}{(x^j-w)^2 + [H - \hat{Z}_{k}]^2}dw + \int_a^b  G_k^j(w^{''}).\hat{Z}_{k}(w) dw^{''} \right\}$$

In [8]:
# sum_int0=0
# sum_int1=0
# partial_sum = 0
# for o in range(len(W)):
#     sum_int1 += np.log(((X[j]-W[o])**2+H**2)/((X[j]-W[o])**2 +(H-Zk_w)**2))
#     sum_int0 += G_k(W[o],X[j],Zk_w,H)
    
# partial_sum = d0[j]-sum_int1 + sum_int0
    

Computation of the matrix S

In [9]:
# # For each Z_k we have to compute the matrix S
# S=np.zeros((len(X),len(X)))
# for p in range(len(X)):
#     for q in range(len(X)):
#         S[p,q]= S_k_ij(G_k,C_d0_d0,C_p0_p0,W,X,Zk_w,H,p,q)
        

### Main 

$$\hat{Z}_{k+1}(w)=\int_a^b \left( \underset{i}{\sum}\underset{j}{\sum}C_{p_0p_0}(w,w').G_k^i(w').(S^{-1})^{ij}.\underset{\text{partial sum}}{\underbrace{\{d_o^j - \int_a^b \text{log}\dfrac{(x^j-w)^2 + H^2}{(x^j-w)^2 + [H - \hat{Z}_{k}]^2}dw + \int_a^b  G_k^j(w^{''}).\hat{Z}_{k}(w) dw^{''} \}}}\right)dw'$$

In [ ]:
for u in range(len(Z)):
    Z_k_w = 0               #for Z_0(w)
    
    ####################### for Z_1(w)
    integral_sum=0
    for r in range(len(W)):      #for the last integral
        sum_i = 0
        for i in range(len(X)):    #for the sum i
            sum_j = 0
            for j in range(len(X)):     #for the sum j
                ###################################### Partial Sum 
                sum_int0=0
                sum_int1=0
                partial_sum = 0
                for o in range(len(W)):
                    sum_int1 += np.log(((X[j]-W[o])**2+H**2)/((X[j]-W[o])**2 +(H-Z_k_w)**2))
                    sum_int0 += G_k(W[o],X[j],Z_k_w,H)

                partial_sum = d0[j]-sum_int1 + sum_int0
                #######################################  Partial Sum  end
                
                ####################################### Matrix S
                S=np.zeros((len(X),len(X)))
                for p in range(len(X)):
                    for q in range(len(X)):
                        S[p,q]= S_k_ij(G_k,C_d0_d0,C_p0_p0,W,X,Z_k_w,H,p,q)
                
                ####################################### Matrix S end

                sum_j += C_p0_p0(W[u],W[r],Sigma,Delta)*G_k(W[r],X[i],Z_k_w,H)*np.linalg.inv(S)[i,j]*partial_sum
            sum_i += sum_j
        integral_sum += sum_i
    Z_k1_w = integral_sum
    ########################
    
    a = 0
    while np.abs(Z_k_w - Z_k1_w) > tolerance or a < nb_iter:
        a += 1
        integral_sum=0
        for r in range(len(W)):      #for the last integral
            sum_i = 0
            for i in range(len(X)):    #for the sum i
                sum_j = 0
                for j in range(len(X)):     #for the sum j
                    ###################################### Partial Sum 
                    sum_int0=0
                    sum_int1=0
                    partial_sum = 0
                    for o in range(len(W)):
                        sum_int1 += np.log(((X[j]-W[o])**2+H**2)/((X[j]-W[o])**2 +(H-Z_k_w)**2))
                        sum_int0 += G_k(W[o],X[j],Z_k_w,H)

                    partial_sum = d0[j]-sum_int1 + sum_int0
                    #######################################  Partial Sum  end

                    ####################################### Matrix S
                    S=np.zeros((len(X),len(X)))
                    for p in range(len(X)):
                        for q in range(len(X)):
                            S[p,q]= S_k_ij(G_k,C_d0_d0,C_p0_p0,W,X,Z_k_w,H,p,q)

                    ####################################### Matrix S end

                    sum_j += C_p0_p0(W[u],W[r],Sigma,Delta)*G_k(W[r],X[i],Z_k_w,H)*np.linalg.inv(S)[i,j]*partial_sum
                    sum_i += sum_j
            integral_sum += sum_i
        Z_k_w = Z_k1_w
        Z_k1_w = integral_sum
    Z[u]= Z_k1_w

In [ ]:
Z